<a href="https://colab.research.google.com/github/AinzOwl/mysticai-colab/blob/main/mystic_sdxl_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate==0.22.0
!pip install diffusers==0.21.4
!pip install safetensors==0.3.3
!pip install torch==2.1.0
!pip install torchvision==0.16.0
!pip install transformers==4.33.1
!pip install xformers==0.0.22.post7
!pip install omegaconf==2.3.0
!pip install pipeline-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.1
    Uninstalling safetensors-0.4.1:
      Successfully uninstalled safetensors-0.4.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.0 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 

In [2]:

#@title Setup Mystic.Ai Api Key

api_key = input('Enter mystic.ai api key: ')
!pipeline cluster login catalyst-api {api_key} -u https://www.mystic.ai -a

Pipeline 00:04:35 - [INFO]: Setting new remote as active 'catalyst-api'
Pipeline 00:04:35 - [INFO]: Successfully authenticated with catalyst-api


In [4]:
from diffusers import AutoPipelineForText2Image, StableDiffusionImg2ImgPipeline
from PIL import Image
import random
import torch
import math
from pathlib import Path
from typing import List

from pipeline import Pipeline, Variable, entity, pipe
from pipeline.cloud import compute_requirements, environments, pipelines
from pipeline.objects import File
from pipeline.objects.graph import InputField, InputSchema

In [17]:
class ModelKwargs(InputSchema):
    turbo_steps: int | None = InputField(
        title="Number of inference steps",
        default=5,
    )
    refiner_steps: int | None = InputField(
        title="Number of inference steps",
        default=15,
    )

    negative_prompt: str | None = InputField(
        default='lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry',
        title="Negative Prompt",
    )

    height: int | None = InputField(
        default=1024,
        title="Height",
        ge=64,
        le=1024,
        multiple_of=8,
    )

    width: int | None = InputField(
        default=1024,
        title="Width",
        ge=64,
        le=1024,
        multiple_of=8,
    )

In [18]:
#@title Setup Model

@entity
class StableDiffusionModel:
    @pipe(on_startup=True, run_once=True)
    def load(self):
       self.txt2img = AutoPipelineForText2Image.from_pretrained(
          "stabilityai/sdxl-turbo",
          torch_dtype = torch.float16,
          variant = "fp16"
       )
       self.txt2img.to("cuda")
       self.img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
          "Lykon/dreamshaper-8",
          torch_dtype = torch.float16,
          variant = "fp16",
          safety_checker=None
       )
       self.img2img.to("cuda")

    @pipe
    def predict(self, prompt: str, kwargs: ModelKwargs) -> List[File]:
        image = self.txt2img(
            prompt=prompt,
            negative_prompt=kwargs.negative_prompt,
            height=kwargs.height,
            width=kwargs.width,
            num_inference_steps=kwargs.turbo_steps,
            guidance_scale=0.0,
            output_type="latent",
        ).images
        images = self.img2img(
            prompt=prompt,
            negative_prompt=kwargs.negative_prompt,
            num_inference_steps=kwargs.refiner_steps,
            guidance_scale=5,
            strength=1,
            image=image,
        ).images

        output_images = []
        for i, image in enumerate(images):
            path = Path(f"/tmp/sd/image-{i}.jpg")
            path.parent.mkdir(parents=True, exist_ok=True)
            image.save(str(path))
            output_images.append(File(path=path, allow_out_of_context_creation=True))

        return output_images

In [19]:
#@title setup mystic environment

with Pipeline() as builder:
    prompt = Variable(
        str,
        title="Prompt",
    )
    kwargs = Variable(
        ModelKwargs,
        title="Model kwargs",
    )

    model = StableDiffusionModel()

    model.load()

    output = model.predict(prompt, kwargs)

    builder.output(output)

my_pl = builder.get_pipeline()

try:
    environments.create_environment(
        "sd-xl-turbo",
        python_requirements=[
            "torch==2.0.1",
            "transformers==4.30.2",
            "diffusers==0.19.3",
            "accelerate==0.21.0",
            "xformers==0.0.21",
            "invisible_watermark==0.2.0",
            "safetensors==0.3.3",
        ],
    )
except Exception:
    pass


Pipeline 00:18:08 - [WARNING]: Package invisible_watermark not found locally, it is highly recommended to install it as this will likely cause serialisation issues when uploading Pipelines.
Pipeline 00:18:08 - [INFO]: Using existing environment sd-xl-turbo with ID = environment_0637a49d9bd74c719ca885ef9a1f58f3


In [20]:

#@title Push the model (open this to change required gpu type)

remote_pipeline = pipelines.upload_pipeline(
    my_pl,
    "sd-xl-turbo:latest",
    environment_id_or_name="sd-xl-turbo",
    required_gpu_vram_mb=20_000,
    accelerators=[
        compute_requirements.Accelerator.nvidia_a100,
    ],
)

print(remote_pipeline.id)

Pipeline 00:18:10 - [SUCCESS]: Uploaded pipeline 'sd-xl-turbo:latest' with ID = pipeline_6a6cd29bee0849da91f06fa99f6ac383
pipeline_6a6cd29bee0849da91f06fa99f6ac383
